In [217]:
import os
import sys
module_path = os.path.abspath(os.path.join('../lstm'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [218]:
from evaluate import *

In [219]:
import os
for f in os.listdir("."):
    if "-" in f and f.endswith(".tsv") and "old" not in f:
        model_name = f.split("-", 1)[-1].replace(".tsv", "")
        evaluator = Evaluator(f, "None")
        results = evaluator.generate_scores_df().round(2)
        print("Results for {}".format(model_name.upper()))
        print(results.to_csv(sep="\t", index=False))

Results for LIN-SVM-L1
Class	P	R	F1	TP	FP	FN
E1 precedes E2	0.49	0.39	0.43	65	68	101
E2 precedes E1	0.0	0.0	0.0	0	4	28
None	0.84	0.91	0.87	652	123	66
MACRO	0.24	0.2	0.22	65	72	129
MICRO	0.47	0.34	0.39	65	72	129

Results for LIN-SVM-L2
Class	P	R	F1	TP	FP	FN
E1 precedes E2	0.53	0.3	0.38	50	45	116
E2 precedes E1	0.25	0.04	0.06	1	3	27
None	0.83	0.94	0.88	674	139	44
MACRO	0.39	0.17	0.22	51	48	143
MICRO	0.52	0.26	0.35	51	48	143

Results for LR-L1
Class	P	R	F1	TP	FP	FN
E1 precedes E2	0.54	0.36	0.43	60	52	106
E2 precedes E1	0.0	0.0	0.0	0	1	28
None	0.84	0.93	0.88	670	129	48
MACRO	0.27	0.18	0.22	60	53	134
MICRO	0.53	0.31	0.39	60	53	134

Results for LR-L2
Class	P	R	F1	TP	FP	FN
E1 precedes E2	0.59	0.27	0.37	44	31	122
E2 precedes E1	0.0	0.0	0.0	0	2	28
None	0.83	0.96	0.89	689	146	29
MACRO	0.29	0.13	0.18	44	33	150
MICRO	0.57	0.23	0.32	44	33	150

Results for RF
Class	P	R	F1	TP	FP	FN
E1 precedes E2	0.57	0.23	0.33	38	29	128
E2 precedes E1	0.5	0.04	0.07	1	1	27
None	0.82	0.96	0.88	688	155	30
MACRO	0.53	0.